In [8]:
from bs4 import BeautifulSoup
import os
import re
import psycopg2 as pg2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import ttest_ind, ttest_ind_from_stats, probplot
import pylab
import statsmodels.stats.api as sms

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

In [2]:
conn = pg2.connect(dbname='dsjob', host='localhost',
                    password='docker', user='postgres')
conn.autocommit = True
cur = conn.cursor()

cur.execute("SELECT * FROM indeed;")
l=[]
for item in cur.fetchall():
    l.append(item)

df=pd.DataFrame(l, columns=["job_jk", "job_title",  "job_location",  "company_name", "salary", "description"])

In [3]:
df.head()

,job_jk,job_title,job_location,company_name,salary,description
0,1bce25f0b7e3723d,"Senior Data Scientist – Image Analytics, Novar...","Cambridge, MA",Novartis,No salary given,\n\nWe believe the answers are found when curi...
1,1486880d275ce7c1,Senior Scientist Deep Data Analytics,"Cambridge, MA 02142",Sanofi,No salary given,Sanofi is a global life sciences company commi...
2,7d6ac302ac846400,Senior Manager of Data Science – Cyber Security,"Boston, MA 02298",No company info,No salary given,\nPrimary Responsibilities:\n Product ownershi...
3,3100f3d85650c469,Enterprise Architect Data,"Boston, MA",MassMutual,No salary given,\nA career with us means you will work alongsi...
4,3de16480e89bcaa2,Post Doctoral Fellow,"Cambridge, MA",Eisai,No salary given,"\n\nAt G2D2, we focus not only on how such dat..."


0        Senior Data Scientist – Image Analytics, Novar...
1                     Senior Scientist Deep Data Analytics
2          Senior Manager of Data Science – Cyber Security
3                               Enterprise Architect  Data
4                                     Post Doctoral Fellow
                               ...                        
14468                                 Postdoctoral Scholar
14469     Staff Machine Learning Engineer  Core Algorithms
14470                          Fullstack Software Engineer
14471                   Managing Editor, Content Marketing
14472              Product Management Intern (Summer 2020)
Name: job_title, Length: 14473, dtype: object


In [17]:
vectorizer = TfidfVectorizer(stop_words = "english", max_features = None)
X = vectorizer.fit_transform(df['description'])
nmf = NMF(n_components=10, random_state =0)
W= nmf.fit_transform(X)
H = nmf.components_

In [18]:
W.shape, H.shape

((14473, 10), (10, 48220))

In [21]:
vocab = vectorizer.get_feature_names()
words_per_topic = []
for i in range(H.shape[0]):
    l=[]
    for e in H[i].argsort()[::-1][0:10]:
        l.append(vocab[e])
    print(l)
    words_per_topic.append(l)
#words_per_topic    

['learning', 'machine', 'data', 'models', 'algorithms', 'ai', 'ml', 'experience', 'science', 'deep']
['product', 'marketing', 'business', 'team', 'customer', 'sales', 'work', 'digital', 'teams', 'experience']
['research', 'university', 'teaching', 'faculty', 'students', 'department', 'position', 'health', 'science', 'edu']
['software', 'experience', 'cloud', 'systems', 'aws', 'development', 'design', 'technical', 'data', 'technologies']
['accenture', 'cloud', 'data', 'client', 'azure', 'visa', 'clients', 'delivery', 'gcp', 'employment']
['deloitte', 'clients', 'people', 'cognitive', 'business', 'client', 'tips', 'learn', 'help', 'offer']
['iqvia', 'healthcare', 'takes', 'clinical', 'reasonable', 'accommodation', 'patients', 'clients', 'health', 'client']
['ll', 'intelligence', 'clearance', 'security', 'data', 'booz', 'allen', 'training', 'access', 'change']
['status', 'employment', 'microsoft', 'protected', 'qualified', 'applicants', 'disability', 'gender', 'information', 'national']
[

In [25]:
def max_feature_topic_search(m_feature_n, topic_n):
    for m in m_feature_n:
        for n in topic_n:
            vectorizer = TfidfVectorizer(stop_words = "english", max_features = m)
            X = vectorizer.fit_transform(df['description'])
            nmf = NMF(n_components=n, random_state =0)
            W= nmf.fit_transform(X)
            H = nmf.components_
            vocab = vectorizer.get_feature_names()
            words_per_topic = []
            print ("max features = {}, n of topics ={}".format(m, n))
            for i in range(H.shape[0]):
                l=[]
                for e in H[i].argsort()[::-1][0:10]:
                    l.append(vocab[e])
                print("topic {}".format(i))
                print(l)
                words_per_topic.append(l)
    

In [26]:
m_feature_n =[20000, 10000, 5000, 2000, 1000]
topic_n = [40, 20, 10, 8, 6, 4]

max_feature_topic_search(m_feature_n, topic_n)

max features = 20000, n of topics =40
topic 0
['reporting', 'bi', 'business', 'data', 'reports', 'analysis', 'ability', 'dashboards', 'tableau', 'sql']
topic 1
['iqvia', 'healthcare', 'takes', 'reasonable', 'accommodation', 'application', 'workday_recruiting', 'brave', 'arrange', 'ultimately']
topic 2
['university', 'teaching', 'faculty', 'students', 'department', 'edu', 'undergraduate', 'graduate', 'school', 'date']
topic 3
['software', 'systems', 'development', 'engineering', 'engineers', 'test', 'code', 'engineer', 'technical', 'infrastructure']
topic 4
['accenture', 'visa', 'client', 'clients', 'employment', 'authorization', 'status', 'delivery', 'technology', 'business']
topic 5
['deloitte', 'people', 'clients', 'cognitive', 'tips', 'learn', 'help', 'confident', 'offer', 'client']
topic 6
['learning', 'machine', 'ai', 'ml', 'algorithms', 'deep', 'models', 'science', 'nlp', 'tensorflow']
topic 7
['intelligence', 'security', 'll', 'clearance', 'warfighters', 'abroad', 'information',

max features = 20000, n of topics =8
topic 0
['data', 'business', 'analytics', 'experience', 'analysis', 'models', 'statistical', 'modeling', 'science', 'tools']
topic 1
['product', 'marketing', 'business', 'team', 'customer', 'sales', 'work', 'strategy', 'teams', 'management']
topic 2
['research', 'university', 'teaching', 'faculty', 'health', 'position', 'department', 'students', 'information', 'program']
topic 3
['experience', 'software', 'cloud', 'learning', 'aws', 'amazon', 'systems', 'machine', 'engineering', 'development']
topic 4
['accenture', 'cloud', 'data', 'azure', 'client', 'visa', 'employment', 'delivery', 'clients', 'status']
topic 5
['deloitte', 'clients', 'people', 'cognitive', 'client', 'learn', 'business', 'tips', 'help', 'offer']
topic 6
['iqvia', 'healthcare', 'takes', 'accommodation', 'reasonable', 'clinical', 'patients', 'clients', 'health', 'client']
topic 7
['ll', 'intelligence', 'clearance', 'security', 'data', 'booz', 'allen', 'training', 'access', 'change']


max features = 10000, n of topics =10
topic 0
['business', 'management', 'ability', 'project', 'analytics', 'skills', 'experience', 'financial', 'analysis', 'work']
topic 1
['product', 'marketing', 'team', 'customer', 'sales', 'products', 'customers', 'business', 'work', 'teams']
topic 2
['research', 'university', 'teaching', 'faculty', 'students', 'department', 'position', 'health', 'science', 'edu']
topic 3
['software', 'experience', 'cloud', 'systems', 'development', 'aws', 'design', 'technical', 'services', 'security']
topic 4
['iqvia', 'healthcare', 'takes', 'accommodation', 'reasonable', 'clinical', 'patients', 'sales', 'health', 'clients']
topic 5
['deloitte', 'people', 'clients', 'cognitive', 'learn', 'tips', 'client', 'help', 'offer', 'confident']
topic 6
['learning', 'machine', 'models', 'algorithms', 'experience', 'ml', 'ai', 'techniques', 'statistical', 'data']
topic 7
['ll', 'intelligence', 'clearance', 'security', 'booz', 'allen', 'access', 'training', 'information', 'dat

max features = 5000, n of topics =20
topic 0
['business', 'analytics', 'management', 'ability', 'analytical', 'skills', 'analysis', 'reporting', 'risk', 'amp']
topic 1
['team', 'work', 'll', 'company', 'people', 'help', 'world', 'best', 'make', 'insurance']
topic 2
['university', 'research', 'teaching', 'faculty', 'students', 'department', 'edu', 'undergraduate', 'graduate', 'position']
topic 3
['software', 'experience', 'development', 'systems', 'design', 'engineering', 'test', 'technical', 'computer', 'web']
topic 4
['iqvia', 'healthcare', 'takes', 'reasonable', 'accommodation', 'clients', 'application', 'workday_recruiting', 'brave', 'client']
topic 5
['deloitte', 'clients', 'people', 'cognitive', 'tips', 'client', 'learn', 'help', 'confident', 'offer']
topic 6
['learning', 'machine', 'models', 'algorithms', 'techniques', 'statistical', 'experience', 'data', 'ml', 'ai']
topic 7
['ll', 'intelligence', 'clearance', 'security', 'booz', 'allen', 'access', 'training', 'change', 'sci']
to

max features = 2000, n of topics =20
topic 0
['data', 'experience', 'analytics', 'sql', 'science', 'sets', 'tools', 'big', 'sources', 'analysis']
topic 1
['university', 'teaching', 'faculty', 'students', 'research', 'department', 'edu', 'undergraduate', 'graduate', 'college']
topic 2
['team', 'll', 'work', 'people', 'company', 'help', 'best', 'world', 'make', 'build']
topic 3
['software', 'experience', 'development', 'systems', 'design', 'engineering', 'test', 'computer', 'web', 'technical']
topic 4
['business', 'analytics', 'management', 'ability', 'analytical', 'skills', 'analysis', 'risk', 'reporting', 'amp']
topic 5
['iqvia', 'healthcare', 'takes', 'reasonable', 'accommodation', 'patients', 'application', 'clients', 'workday_recruiting', 'brave']
topic 6
['status', 'employment', 'protected', 'gender', 'qualified', 'microsoft', 'disability', 'applicants', 'origin', 'race']
topic 7
['deloitte', 'clients', 'people', 'cognitive', 'client', 'tips', 'learn', 'help', 'confident', 'offer']

max features = 1000, n of topics =20
topic 0
['data', 'big', 'science', 'pipelines', 'sql', 'analytics', 'etl', 'tools', 'sets', 'sources']
topic 1
['university', 'teaching', 'faculty', 'students', 'department', 'edu', 'graduate', 'undergraduate', 'college', 'position']
topic 2
['team', 'll', 'work', 'people', 'company', 'help', 'world', 'best', 'make', 'love']
topic 3
['software', 'experience', 'development', 'design', 'systems', 'engineering', 'code', 'test', 'web', 'testing']
topic 4
['learning', 'machine', 'ai', 'models', 'ml', 'algorithms', 'deep', 'science', 'data', 'techniques']
topic 5
['iqvia', 'healthcare', 'takes', 'accommodation', 'reasonable', 'application', 'patients', 'clients', 'impact', 'disabled']
topic 6
['status', 'employment', 'gender', 'protected', 'disability', 'qualified', 'applicants', 'origin', 'race', 'religion']
topic 7
['deloitte', 'clients', 'people', 'client', 'learn', 'help', 'offer', 'professionals', 'culture', 'communities']
topic 8
['statistical', 'an